### CNN-based 3D Skeleton Tennis Shot Recognition

Based on method from paper ['Skeleton-based Action Recognition with Convolutional Neural Networks'](https://arxiv.org/abs/1704.07595) by Li et al. (2017).

We use the 3D skeletal animation data from the ['THETIS'](http://thetis.image.ece.ntua.gr/) dataset for training. Each animation is represented as a `T x N` 3-channel image where `T` is the number of frames of the animation, `N` is the number of skeletal joints, and the 3 colour channels represent `x, y, z` coordinates of the joints.

Load the skeleton images training data.

In [25]:
import tensorflow as tf

skeleton_dir = "data/skeleton_images"

img_height = 250
img_width = 15
batch_size = 32

skeleton_train_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

skeleton_val_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 621 files belonging to 24 classes.
Using 497 files for training.
Found 621 files belonging to 24 classes.
Using 124 files for validation.


Create the CNN model.

In [26]:
from tensorflow.keras import layers, models

model = models.Sequential()
model.add(layers.Rescaling(1./127.5, offset=-1))
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(250,15,3)))
model.add(layers.MaxPooling2D((2,1)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,1)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(24, activation='softmax'))


Compile and train the model.

In [27]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  skeleton_train_ds,
  validation_data=skeleton_val_ds,
  epochs=100
)

model.save('cnn_recognition_model.h5')

Epoch 1/100
16/16 [==============================] - 2s 125ms/step - loss: 3.2191 - accuracy: 0.0362 - val_loss: 3.1727 - val_accuracy: 0.0726
Epoch 2/100
16/16 [==============================] - 2s 116ms/step - loss: 3.1533 - accuracy: 0.0604 - val_loss: 3.1745 - val_accuracy: 0.0403
Epoch 3/100
16/16 [==============================] - 2s 110ms/step - loss: 3.0734 - accuracy: 0.1087 - val_loss: 3.1767 - val_accuracy: 0.0484
Epoch 4/100
16/16 [==============================] - 2s 110ms/step - loss: 2.9152 - accuracy: 0.1489 - val_loss: 3.2004 - val_accuracy: 0.0645
Epoch 5/100
16/16 [==============================] - 2s 108ms/step - loss: 2.6641 - accuracy: 0.2113 - val_loss: 3.2202 - val_accuracy: 0.1290
Epoch 6/100
16/16 [==============================] - 2s 109ms/step - loss: 2.3734 - accuracy: 0.2877 - val_loss: 3.1899 - val_accuracy: 0.1290
Epoch 7/100
16/16 [==============================] - 2s 107ms/step - loss: 2.0319 - accuracy: 0.3883 - val_loss: 3.2623 - val_accuracy: 0.1774

KeyboardInterrupt: 